In [22]:
import requests

r = requests.get('https://dadosabertos.bcb.gov.br/api/rest/dataset/dolar')
r.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
%load_ext nb_black

In [214]:
from collections import OrderedDict

import defusedxml.cElementTree as ET
import pandas as pd
from xml.etree import ElementTree
import jinja2 as j2
import sgs
import xml.dom.minidom

In [215]:
template = """
<?xml version="1.0" encoding="utf-8"?>
      <soapenv:Envelope
        xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/"
        xmlns:xsd="http://www.w3.org/2001/XMLSchema"
        xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
        <soapenv:Body>
          <{{ method }}
            xmlns="https://www3.bcb.gov.br/wssgs/services/FachadaWSSGS">
            {%- for p in params -%}
              {%- if p == "codigosSeries" %}
                <{{ p }}>
                  <item>{{ params[p] }}</item>
                </{{ p }}>
              {%- else %}
                <{{ p }}>{{ params[p] }}</{{ p }}>
              {%- endif %}
            {%- endfor %}
          </{{ method }}>
        </soapenv:Body>
      </soapenv:Envelope>
""".strip()

In [216]:
tp = j2.Template(template)

In [217]:
method="getValoresSeriesXML"
params = OrderedDict(
    [("codigosSeries", 12), ("dataInicio", "01/01/2018"), ("dataFim", "01/02/2018")]
)
tp_str = tp.render(params=params, method="getValoresSeriesXML")
print(tp_str)

<?xml version="1.0" encoding="utf-8"?>
      <soapenv:Envelope
        xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/"
        xmlns:xsd="http://www.w3.org/2001/XMLSchema"
        xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
        <soapenv:Body>
          <getValoresSeriesXML
            xmlns="https://www3.bcb.gov.br/wssgs/services/FachadaWSSGS">
                <codigosSeries>
                  <item>12</item>
                </codigosSeries>
                <dataInicio>01/01/2018</dataInicio>
                <dataFim>01/02/2018</dataFim>
          </getValoresSeriesXML>
        </soapenv:Body>
      </soapenv:Envelope>


In [270]:
sgs.series.BOVESPA_VOLUME

8

In [278]:
sgs.series.INCC

192

In [279]:
template1 = """
<?xml version="1.0" encoding="utf-8"?>
      <soapenv:Envelope
        xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/"
        xmlns:xsd="http://www.w3.org/2001/XMLSchema"
        xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
        <soapenv:Body>
          <getValoresSeriesXML
            xmlns="https://www3.bcb.gov.br/wssgs/services/FachadaWSSGS">
                <codigosSeries>  
                  <item>8</item>
                  <item>11</item>                  
                  <item>192</item>
                </codigosSeries>
                <dataInicio>01/01/2018</dataInicio>
                <dataFim>01/03/2018</dataFim>
          </getValoresSeriesXML>
        </soapenv:Body>
      </soapenv:Envelope>
""".strip()
sgs.sgs.template = template1

In [280]:
def extract_series_tree(api_response: bytes) -> ElementTree.Element:
    tree = ET.fromstring(api_response)
    XML = list(tree.iter(tag="getValoresSeriesXMLReturn"))[0].text
    series_tree = ET.fromstring(XML)
    return series_tree

In [281]:
def parse_serie_xml(serie: ElementTree.Element) -> pd.Series:
    name = serie.attrib.get("ID", "")    
    index = [i.text for i in serie.iter('DATA')]
    index = pd.to_datetime(index, dayfirst=True)
    data = [i.text for i in serie.iter('VALOR')]
    return pd.Series(data=data, index=index, name=name)   

In [ ]:
codigo_serie = 192
dataInicial = '01/01/2018'
dataFinal = '01/03/2019'

In [36]:
pd.read_csv?

In [75]:
import pandas as pd
def get_data(codigo_serie: list, data_inicial: str, data_final: str):        
    series = []
    for cod in codigo_serie:
        url = f"http://api.bcb.gov.br/dados/serie/bcdata.sgs.{cod}/dados?formato=csv&dataInicial={data_inicial}&dataFinal={data_final}"
        df = pd.read_csv(url, pytesindex_col='data', parse_dates=True, dayfirst=True, sep=';', names=['data', cod], header=0)
        del df.index.name
        series.append(df)
    return pd.concat(series, axis=1)
    

df = get_data([12, 191], '02/01/2018', '01/04/2018')
df

,12,191
2018-01-01,NaN,"0,69"
2018-01-02,"0,026444",NaN
2018-01-03,"0,026444",NaN
2018-01-04,"0,026444",NaN
2018-01-05,"0,026444",NaN
2018-01-08,"0,026444",NaN
2018-01-09,"0,026444",NaN
2018-01-10,"0,026444",NaN
2018-01-11,"0,026444",NaN
2018-01-12,"0,026444",NaN


In [282]:
bc = sgs.SGS()
api_response = bc.requests_wssgs(method=method, params=params)

HTTPError: 500 Server Error: Internal Server Error for url: https://www3.bcb.gov.br/wssgs/services/FachadaWSSGS?method=getValoresSeriesXML

In [275]:
series = []
series_tree = extract_series_tree(api_response)
print(series_tree)

<Element 'SERIES' at 0x7f5787020bd8>


In [276]:
series = []
series_data = series_tree.iter(tag='SERIE')
for i in series_data:
    print(i)
    s = parse_serie_xml(i)
    series.append(s)

<Element 'SERIE' at 0x7f5787020c28>
<Element 'SERIE' at 0x7f5786faaea8>


In [277]:
pd.concat(series, axis=1)

,8,11
2018-01-02,8002,0.026481
2018-01-03,8740,0.026481
2018-01-04,9617,0.026481
2018-01-05,7998,0.026481
2018-01-08,7399,0.026481
2018-01-09,9300,0.026481
2018-01-10,7751,0.026481
2018-01-11,8609,0.026481
2018-01-12,9146,0.026481
2018-01-15,10167,0.026481
